## A Notebook to get Information from existing IIIF-Manifests, select canvases, and arrange them into a new manifest


NOTE: This will only work with existing manifests following the version 3 of the IIIF Presentation API.

In [1]:
import requests
import json

In [2]:
# function to create a new manifest
def create_manifest(manifest_id, label, metadata, summary, provider):
   
    new_manifest = {
        "@context": "http://iiif.io/api/presentation/3/context.json",
        "id": manifest_id,
        "type": "Manifest",
        "label": label,
        "summary": summary,
        "metadata": metadata,
        "provider": provider,
        "items": []  # No canvases yet
    }  

    return new_manifest

In [13]:
# Provide basic Metadata for the new manifest
base_url="https://raw.githubusercontent.com/r0man-ist/iiif/refs/heads/main/"
filename ="test.json"
manifest_id=base_url+filename ## required
label={"de": ["Hensel, Wilhelm/Fanny Hensel: Briefsequenz aus der Verlobungszeit 1829"]} ## required
metadata=[
    {
         "label": "Autor",
         "value": ["Hensel, Fanny", "Hensel, Wilhelm"]
    },
    {
         "label": "Titel",
         "value": "Briefsequenz aus der Verlobungszeit von Fanny und Wilhelm Hensel, 1829"
    },
    {
      "label": "Signaturen",
      "value": ["MA Depos. 3,6", "MA Depos. 3,4"]
    },  
    {
      "label": "Sprache",
      "value": "ger"
    },
     {
      "label": "Lizenz",
      "value": "Public Domain Mark 1.0"
    },
        ]
summary = { "de": [ "Bei diesem virtuellen Objekt handelt es sich um eine Briefsequenz aus ..." ] }
provider = [
    {
      "id": "https://lab.sbb.berlin",
      "type": "Agent",
      "label": { "en": [ "Stabi Lab" ] },
      "homepage": [
        {
          "id": "https://lab.sbb.berlin",
          "type": "Text",
          "label": { "de": [ "Stabi Lab Homepage" ] },
          "format": "text/html"
        }]
    }      
  ]
    
new_manifest = create_manifest(manifest_id, label, metadata, summary, provider)

In [14]:
# function to get manifests from a list of urls
def fetch_manifests(manifest_urls):
    manifests = []
    for url in manifest_urls:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        manifest = response.json()  # Parse the JSON response
                             
        manifests.append(manifest)
            
    return manifests

In [15]:
# specify a list of manifests to fetch
manifest_urls = ["https://raw.githubusercontent.com/r0man-ist/iiif/refs/heads/main/1880377578v3.manifest.json"]
manifests = fetch_manifests(manifest_urls)
print(manifests)



[{'@context': ['http://www.w3.org/ns/anno.jsonld', 'http://iiif.io/api/presentation/3/context.json'], 'id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578/manifest', 'type': 'Manifest', 'label': {'true': ['Hensel, Wilhelm: Briefe und Billets an Fanny Hensel aus der Verlobungszeit 1829']}, 'metadata': [{'label': {'true': ['Autor']}, 'value': {'true': ['Hensel, Wilhelm']}}, {'label': {'true': ['Titel']}, 'value': {'true': ['Briefe und Billets an Fanny Hensel aus der Verlobungszeit 1829']}}, {'label': {'true': ['Projekt']}, 'value': {'true': ['NachlÃ¤sse und Autographe digital']}}, {'label': {'true': ['_digitalOrigin']}, 'value': {'true': ['reformatted digital']}}, {'label': {'true': ['_electronicEdition']}, 'value': {'true': ['[Electronic ed.]']}}, {'label': {'true': ['_placeOfElectronicOrigin']}, 'value': {'true': ['Berlin']}}, {'label': {'true': ['Bemerkung']}, 'value': {'true': ['P_SBB_Sondermat_Nachlaesse']}}, {'label': {'true': ['Bemerkung']}, 'value': {'true': ['108 Brie

In [16]:
# get individual canvases from the manifests
canvases = []
for manifest in manifests:  
    for canvas in manifest.get('items', []):
        if canvas.get('type') == 'Canvas':
            canvases.append(canvas)

# Print the extracted canvases
print(canvases)

[{'id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001/canvas', 'type': 'Canvas', 'label': {'true': ['1r [1]']}, 'height': 900, 'width': 600, 'items': [{'id': 'https://example.org/uuid/f08c37ad-4ea8-4262-b8ed-4198afd68f32', 'type': 'AnnotationPage', 'items': [{'id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001/annotation', 'type': 'Annotation', 'target': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001/canvas', 'body': {'id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001/full/full/0/default.jpg', 'type': 'Image', 'service': [{'@id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001', '@type': 'ImageService2', 'profile': 'http://iiif.io/api/image/2/level1.json'}]}}]}]}, {'id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0002/canvas', 'type': 'Canvas', 'label': {'true': ['1v [2]']}, 'height': 900, 'width': 600, 'items': [{'id': 'https://example.org/uuid/dab85952-d1af-4e1a-a459-82897611f583', 'type

In [17]:
# specify a list of canvases to include in the new manifest
# canvases will be included in the order in which they appear in the list
canvas_list = [
    "https://content.staatsbibliothek-berlin.de/dc/1880377578-0243/canvas",
    "https://content.staatsbibliothek-berlin.de/dc/1880377578-0242/canvas",
    "https://content.staatsbibliothek-berlin.de/dc/1878144588-0032/canvas"
]


In [18]:
matching_canvases = []

# Create a dictionary for quick lookup using the canvas IDs
canvas_ids = {canvas["id"]: canvas for canvas in canvases}

# Iterate over the canvas_list to maintain the specified order
for canvas_id in canvas_list:
    if canvas_id in canvas_ids:
        matching_canvases.append(canvas_ids[canvas_id])

new_manifest["items"] = matching_canvases

In [20]:

with open("test.json", "w", encoding="utf-8") as f:
        json.dump(new_manifest, f, ensure_ascii=False, indent=4)

In [ ]:
# Not needed!
# rename keys to comply with V3 of the IIIF Presentation API
for item in new_manifest["items"]:
    if "@type" in item and item["@type"] == "sc:Canvas":
        item["type"] = "canvas"  # Change to "type"
        del item["@type"]  # Optionally remove the old key